**Pytorch to Tflite via Keras**

First upload the '**model_mobilenetv2_seg_small.py**' file from portrait-net repo into colab root folder i.e **'/content'**. It contains the **architecture** of the model. Now upload the **weights** of the trained model i.e '**pnet_video.pth**' into colab.

In [0]:
import torch
import torchvision
import model_mobilenetv2_seg_small

dummy_input = torch.randn(1, 4, 224, 224).cuda()
model =  torch.load('/content/pnet_video.pth').cuda()

torch.onnx.export(model, dummy_input, "pnet_video.onnx", verbose=True)

**Note:** Providing input and output names sets the display names for values within the model's graph. Setting these does not change the semantics of the graph; it is only for readability.

The inputs to the network consist of the flat list of inputs (i.e. the values you would pass to the forward() method) followed by the flat list of parameters. You can partially specify names, i.e. provide a list here shorter than the number of inputs to the model, and we will only set that subset of names, starting from the beginning.

You may also use the **entire saved model**, including model architecture and weights, as input to the converter.

**ONNX to Keras**

Install latest version of **tensorflow, onnx and onnx2keras**.

In [0]:
!pip install tensorflow-gpu
!pip install onnx
!git clone https://github.com/nerox8664/onnx2keras.git

Change directory to **onnx2keras** root folder.

In [0]:
%cd onnx2keras

Load the onnx model and convert it to keras model with **onnx2keras**. Ensure that the parameter **change_ordering** is True, for changing the channel format from **NCHW to NHWC**.

In [0]:
import tensorflow as tf
import onnx
from onnx2keras import onnx_to_keras
from tensorflow.keras.models import load_model

# Load ONNX model
onnx_model = onnx.load('/content/pnet_video.onnx')

# Call the converter and save keras model
k_model = onnx_to_keras(onnx_model, ['input.1'],change_ordering=True)
k_model.save('/content/pnet_video.h5')

**NB:** It may take about a minute for the conversion process to complete. 

**Keras Model Modification**

Our model contains **two outputs** corresponding to mask and edge. We need to **remove** the **edge output** from the model, since it is not need during model inference.

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Activation, Lambda, Reshape

In [0]:
# Load keras model
k_model=load_model('/content/pnet_video.h5')
k_model.summary()

In [0]:
# Remove edge branch from output
edge_model=Model(inputs=k_model.input,outputs=k_model.layers[-2].output)
edge_model.summary()

Now our model has one **two channel** output corresponding to foreground and output. First add a **softmax layer** at the end of the model(over channel axis) to restrict  the output range between 0 and 1.

In [0]:
# Add softmax on output
sm=Lambda(lambda x: tf.nn.softmax(x))(edge_model.output)
soft_model=Model(inputs=edge_model.input, outputs=sm)
soft_model.summary()

Now, let's get the softmax slice for the **foreground** channel, using **strided slice**.

In [0]:
# Get foreground softmax slice
ip = soft_model.output
str_slice=Lambda(lambda x: tf.strided_slice(x, [0,0, 0, 1], [1,224, 224, 2], [1, 1, 1, 1]))(ip)
stride_model=Model(inputs=soft_model.input, outputs=str_slice)
stride_model.summary()

Finally, **flatten** the output to 1D i.e 1x224x22x1 => 1x50176

In [0]:
# Flatten output
output = stride_model.output
newout=Reshape((50176,))(output)
reshape_model=Model(stride_model.input,newout)
reshape_model.summary()

Save the **final keras** model.

In [0]:
# Save keras model
reshape_model.save('/content/portrait_video.h5')

**Keras to Tflite**

Finally, convert the keras model to tflite using tensorflow **tflite-converter**.

In [0]:
# Convert to tflite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(reshape_model)
tflite_model = converter.convert()
open("/content/portrait_video.tflite", "wb").write(tflite_model)

**NB:** For verification or model inspection use [Netron](https://lutzroeder.github.io/netron/) web-app.